In [1]:
!pip install transformers[torch] datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-p

In [2]:
from transformers import AutoTokenizer, BertForTokenClassification

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

# Load model phoBERT sau khi đã finetune để tìm vị trí lỗi

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import zipfile

zip_path = '/content/gdrive/MyDrive/checkpoint-6000.zip'
extract_path = 'checkpoint'

# Mở tệp zip và giải nén
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [5]:
pretrain_checkpoint = 'checkpoint'
pretrain_model = BertForTokenClassification.from_pretrained(pretrain_checkpoint)
pretrain_model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05

# Xác định vị trí lỗi

In [6]:
import torch
import numpy as np

# Input: 1 câu bị sai chính tả, show = True nếu muốn in ra từ bị lỗi
# Output: list index của các từ dự đoán là lỗi, mask của câu sau khi dự đoán (0 - Từ đúng, 1 - Từ đơn sai, 2 - Từ sai do xóa dấu cách)
def GetErrorIndex(sentence, show = False):
    inputs = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    with torch.no_grad():
        outputs = pretrain_model(**inputs)

    output = np.argmax(outputs.logits, axis=-1).reshape(-1)
    pred = []
    temp = []
    for i, token in enumerate(tokens):
        # if i > 0 and show:
        #     print(token, output[i].item())
        if token not in ['<s>', '<unk>', '<pad>', '</s>']:
            temp.append(output[i].item())
            if token[-2:] != '@@':
                pred.append(round(sum(temp)/len(temp) +0.1))
                temp.clear()
    pred = np.array(pred)

    # Loại trường hợp một số bị dự đoán là sai -----vì số liệu thì k biết đúng hay sai------------------
    words = sentence.split()
    for _, word in enumerate(words):
        if pred[_] and word.isdigit(): pred[_] = 0

    if show:
      print("Error words: ", end = '')
      for err, word in zip(pred, sentence.split()):
        if err: print(word, end=', ')
      print()
    return np.where(pred != 0)[0], pred


In [7]:
sentence = 'Ra leejnh tập trận bằng vũ khí hạtnhân chến thuâtj , ông Putin dường mhư muốn thểiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine.'
GetErrorIndex(sentence, True)

Error words: leejnh, hạtnhân, chến, thuâtj, mhư, thểiện, Ukraine., 


(array([ 1,  7,  8,  9, 14, 16, 32]),
 array([0, 1, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

# Masked LM

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

model = AutoModelForMaskedLM.from_pretrained('vinai/phobert-base-v2')

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [9]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=ce78357871247768854879a2bdd7e1bee16e1a044e5dc15793142d8b7fbf8750
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [10]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='/content')

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [11]:
# Input: 1 câu sai chính tả
# Output: Một list chứa các list top các từ dự đoán cho mỗi vị trí đc <mask>
def phobert_MaskedLM(sentence, top = 10, show = False):
  # word segment------------------------
  # segment_sent = rdrsegmenter.word_segment(sentence)[0].replace('< mask >', '<mask>')
  segment_sents = rdrsegmenter.word_segment(sentence)
  segment_sent = " ".join(segment_sents).replace('< mask >', '<mask>') # Phòng trường hợp câu bị tách thành nhiều câu riêng
  if show:
    print(segment_sent)

  # tokenize ------------------------------
  inputs = tokenizer(segment_sent, return_tensors="pt")
  with torch.no_grad():
      logits = model(**inputs).logits
  mask_token_indexs = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

  # get top predict ------------------------
  top_predict = []
  for i, mask_idx in enumerate(mask_token_indexs):
    cur_pred = []
    # print(f"Dự đoán <mask> thứ {i + 1} --------------")
    top_predicted_token_ids = torch.topk(logits[0, mask_idx], k=top).indices.reshape(-1)
    for _, token_id in enumerate(top_predicted_token_ids):
      # print(f'Top {_ + 1}: ', tokenizer.decode([token_id]))
      cur_pred.append(tokenizer.decode([token_id]))

    top_predict.append(cur_pred)
  return top_predict

sentence = 'Ông Nguyễn Khắc Chúc đang làm việc tại đại <mask> Quốc gia Hà Nội .'
phobert_MaskedLM(sentence, 20)


[['hoc',
  'Đại_học',
  'học',
  'Thư_viện',
  'học_tập',
  'Học_@@',
  'Chính_trị',
  'chủng_viện',
  'đại_học',
  'điện',
  'trường',
  'Âm_nhạc',
  'ĐH',
  'Đại_Học',
  'ich',
  'Sứ_quán',
  'lí_do',
  'học_t@@',
  'chính_trị',
  'viện']]

## BARTPho

In [12]:
import torch
from transformers import AutoModel, AutoTokenizer, MBartForConditionalGeneration

#BARTpho-syllable
bartpho_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
bartpho_model = MBartForConditionalGeneration.from_pretrained("vinai/bartpho-syllable")

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [13]:
# Input: 1 câu sai chính tả
# Output: Một list chứa các list top các từ dự đoán cho mỗi vị trí đc <mask>
def bartpho_MaskedLM(sentence, top = 10, show = False):
  # tokenize ------------------------------
  inputs = bartpho_tokenizer(sentence, return_tensors="pt")
  with torch.no_grad():
      logits = bartpho_model(**inputs).logits
  mask_token_indexs = (inputs.input_ids == bartpho_tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

  # get top predict ------------------------
  top_predict = []
  for i, mask_idx in enumerate(mask_token_indexs):
    cur_pred = []
    # print(f"Dự đoán <mask> thứ {i + 1} --------------")
    top_predicted_token_ids = torch.topk(logits[0, mask_idx], k=top).indices.reshape(-1)
    for _, token_id in enumerate(top_predicted_token_ids):
      # print(f'Top {_ + 1}: ', bartpho_tokenizer.decode([token_id]))
      cur_pred.append(bartpho_tokenizer.decode([token_id]))

    top_predict.append(cur_pred)
  return top_predict

sentence = 'Ra lệnh tập trận bằng vũ khí <mask> <mask> chiến thuật , ông Putin dường <mask> muốn thể hiện nỗi giận dữ'
bartpho_MaskedLM(sentence, 20)


[['hạt',
  'thông',
  'tối',
  'chính',
  'hu',
  'chiến',
  'hiện',
  'mới',
  'nguyên',
  'và',
  'hạng',
  'tấn',
  'công',
  'mạnh',
  'hoá',
  'phòng',
  'không',
  'sát',
  ',',
  'quân'],
 ['-',
  'Ele',
  'Hammer',
  ',',
  'Ar',
  'Kat',
  '<unk>',
  'Z',
  'Hel',
  'Ton',
  'Bla',
  'V',
  'To',
  'Né',
  'để',
  '',
  'No',
  'a',
  'và',
  'ar'],
 ['như',
  'đang',
  'không',
  'chỉ',
  'muốn',
  'nghĩ',
  'có',
  'đã',
  'chừng',
  't',
  'n',
  'chẳng',
  'd',
  'nhưng',
  'hồ',
  'nào',
  'cũng',
  'dư',
  'đây',
  'đó']]

In [40]:
def SpellingCorrection_type2(err_word): # kiểm tra xem từ này có phải lỗi do xóa space không
    # Tokenize the word
    tokens = tokenizer.tokenize(err_word)
    # print(tokens)
    new_word = err_word[:-len(tokens[-1])] + ' ' + tokens[-1]
    # print(new_word)
    err_ids, masked = GetErrorIndex(new_word)
    if len(err_ids) == 0:
      return True, err_word[:-len(tokens[-1])] + '_' + tokens[-1]
    else:
      return False, err_word

SpellingCorrector_type2('mởrộng')

(True, 'mở_rộng')

# So sánh độ tương đồng giữa từ dự đoán và từ lỗi

In [41]:
!pip install unidecode
!pip install Levenshtein

In [42]:
import Levenshtein
from unidecode import unidecode

def distance(pred, err, del_tone = True):
  if del_tone:
      pred = unidecode(pred)
      err = unidecode(err)

  sc = 0 # Số kí tự có trong từ lỗi
  for c in pred:
    if c in err: sc += 1
  return Levenshtein.distance(pred, err) - sc # score càng nhỏ thì càng tốt

distance('buồl', 'tiếc')

4

In [43]:
def SpellingCorrection_1sent(sentence, MaskedLM = bartpho_MaskedLM, top = 15, show = False):
  # Xác định vị trí lỗi
  err_idx, pred = GetErrorIndex(sentence)
  # print(err_idx, pred)
  words = sentence.split()

  # Sửa lỗi loại 1 ---------------------------------------------
  # Duyệt qua từng vị trí lỗi và mask nó lại -> sau đó dự đoán. Tiếp tục như vậy tới vị trí lỗi tiếp theo.
  for idx in err_idx:
    if pred[idx] != 2:
      err_word = words[idx]
      words[idx] = '<mask>'
      masked_sentence = ' '.join(words)

      top_pred = MaskedLM(masked_sentence, top=top)
      # print(masked_sentence)
      # print(top_pred)
      # print()
      dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
      for pred_word in top_pred[0]:
        if '_' not in pred_word and '@@' not in pred_word: # Lỗi loại 1 chỉ là từ đơn
          dist.append(distance(pred_word, err_word))
        else:
          dist.append(1000)

      new_word_id = np.argmin(np.array(dist))
      new_word = top_pred[0][new_word_id]
      words[idx] = new_word
      if show:
        print( 'err_type 1: ' + err_word + " --->>> " + new_word)

  # Sửa lỗi loại 2
  # 2.1 -- Sửa lỗi bị xóa space
  masked = [0] * len(words)
  for idx in err_idx:
    if pred[idx] == 2:
      err_word = words[idx]
      chk, new_word = SpellingCorrection_type2(err_word)
      if chk:
        words[idx] = new_word # Chưa kiểm tra xem còn lỗi khác ngoài xóa space không
        masked[idx] = 1
      if show:
        print( 'err_type 2: ' + err_word + " --->>> " + new_word , chk)
  new_sentence2 = ' '.join(words)

  # 2.2 -- Sửa lỗi bị xóa space và các lỗi khác nữa
  for idx in err_idx:
    if pred[idx] == 2 and masked[idx] == 0:
      err_word = words[idx]
      words[idx] = '<mask> <mask>'
  masked_sentence = ' '.join(words).replace('_', ' ')
  top_pred = MaskedLM(masked_sentence, top=top)
  # print(top_pred)

  # mask tất cả  -------------------------------
  words = new_sentence2.split()
  cnt = 0
  for idx in err_idx:
     if pred[idx] == 2 and masked[idx] == 0:
        err_word = words[idx]
        words[idx] = ""

        dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
        _pred = top_pred[cnt] + top_pred[cnt + 1]

        for pred_word in _pred:
          if len(pred_word.split('_')) == 2:
            dist.append(distance(pred_word.replace('_', ''), err_word))
          else:
            dist.append(1000)

        new_word_id = np.argmin(np.array(dist))
        new_word = _pred[new_word_id]

        if len(new_word.split('_')) == 2:
          words[idx] = new_word
          cnt += 2
          if show:
            print( 'err_type 2: ' + err_word + " --->>> " + words[idx])
          continue

        dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
        for pred_word in top_pred[cnt]:
          dist.append(distance(pred_word, err_word))
        new_word_id = np.argmin(np.array(dist))
        new_word = top_pred[cnt][new_word_id]
        words[idx] += new_word + '_'
        cnt += 1

        dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
        for pred_word in top_pred[cnt]:
          dist.append(distance(pred_word, err_word))
        new_word_id = np.argmin(np.array(dist))
        new_word = top_pred[cnt][new_word_id]
        words[idx] += new_word
        if show:
          print( 'err_type 2: ' + err_word + " --->>> " + words[idx])
        cnt += 1

  #   # mask lần lượt  -------------------------------

  return " ".join(words).replace('_', ' ')

In [45]:
sentence = 'Dự án đầu tư cari tạo , mởrộngg vành đai 2 trên cao vàa dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giấy có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .'
SpellingCorrection_1sent(sentence, phobert_MaskedLM, show = True)

err_type 1: cari --->>> cải
err_type 1: vàa --->>> và
err_type 1: TƯ --->>> Tư
err_type 1: Cầu --->>> Cầu
err_type 1: Giấy --->>> Giẽ
err_type 2: mởrộngg --->>> mởrộngg False
err_type 2: mởrộngg --->>> mở_rộng


'Dự án đầu tư cải tạo , mở rộng vành đai 2 trên cao và dưới thấp đoạn từ ngã Tư Sở đến Cầu Giẽ có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .'

In [46]:
import re
def SpellingCorrection(text, MaskedLM = bartpho_MaskedLM, top = 20, show = False):
  # Tiền xử lý text
  tokens = re.findall(r'\w+|[^\w\s]', text) # Tách dấu câu ra
  text = " ".join(tokens)
  return SpellingCorrection_1sent(text, MaskedLM, top)

sentence = 'Ra leejnh tập trận bằng vũkhí hạt nhân chien thuat , ông Putin đường mhư muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .'
SpellingCorrection(sentence)

'Ra lệnh tập trận bằng vũ khí hạt nhân chiến thuật , ông Putin đường như muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .'

In [48]:
sentence = ['Ra leejnh tập trận bằng vũkhí hạt nhân chien thuat , ông Putin đường mhư muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .',
  'HàNội sẽ mở rộng đường Láng từ khoảng 21 m cả haii Chiều hiennay lên 53 , 5 m , đồng thời xxaay dựng đường trên caoo từ Ngã Tư Sở đến Cầu Giấy .',
  'Sở Giao thông Vận tải vừa có báo cáo UBND thành phố việc triển khai lập báo cáo đề xuất chủtrương đầu tư',
  'Dự án đầu tư cải tạo , mở rộng vanh dai 2 trên cao và dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giấy có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .',
  'campuchia tuyên bố không tham vần thêm về kếnh Funan Techo',
  'TP HCM sẽ làm tuyếN đường " mẫu " về k gian đô thị ở khu trung tâm',
   'Loại PSG , Dortmund vào chung kết Champions League',
  '500 cổ vật quý hiếm , trongđó có 18 bảo vật quốcgia sẽ được trưng bày tại Bảo tàngg Hải Phòng để người dân tham quan từ ngày 11 / 5 . ',
  'Nỗi khoo chủ quán cà phee gặp khách ngồi " cắm rễ "',]

for sent in sentence:
  print(sent)
  print("câu đã sửa: ", SpellingCorrection(sent, bartpho_MaskedLM))
  print()

Ra leejnh tập trận bằng vũkhí hạt nhân chien thuat , ông Putin đường mhư muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .
câu đã sửa:  Ra lệnh tập trận bằng vũ khí hạt nhân chiến thuật , ông Putin đường như muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .

HàNội sẽ mở rộng đường Láng từ khoảng 21 m cả haii Chiều hiennay lên 53 , 5 m , đồng thời xxaay dựng đường trên caoo từ Ngã Tư Sở đến Cầu Giấy .
câu đã sửa:  Hà Nội sẽ mở rộng đường Láng từ khoảng 21 m cả hai  Chiều ngang hiện lên 53 , 5 m , đồng thời xây dựng đường trên cao từ Ngã Tư Sở đến Cầu Giấy .

Sở Giao thông Vận tải vừa có báo cáo UBND thành phố việc triển khai lập báo cáo đề xuất chủtrương đầu tư
câu đã sửa:  Sở Giao thông Vận tải vừa có báo cáo UBND thành phố việc triển khai lập báo cáo đề xuất chủ trương đầu tư

Dự án đầu tư cải tạo , mở rộng vanh dai 2 trên cao và dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giấy có tổng đầu tư trên 21 . 000 tỷ 